# Imports

In [ ]:
# general imports
import sys                       
import numpy as np
import time

# gym for spaces and environment definitions
import gym
# env checker
from stable_baselines3.common import env_checker

# find nan and inf values propagating quietly in the network
from stable_baselines3.common.vec_env import DummyVecEnv, VecCheckNan

# stable baselines3 -> TD3
from stable_baselines3 import TD3

# stbale baselines3 -> HER
from stable_baselines3.her.her_replay_buffer import HerReplayBuffer
from stable_baselines3.her.goal_selection_strategy import GoalSelectionStrategy

# Tensorboard
from torch.utils.tensorboard import SummaryWriter

# grpc communication
sys.path.insert(1, '/tum_nrp/grpc/python/communication')
import experiment_api_wrapper as eaw

# for auto-reloading external modules see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from env import SimEnv, SimGoalEnv
from train_helpers import evaluate3, train

# Image Data

In [ ]:
# for path minipulation
import os

# tensor Operations
import torch

# custom dataset class for the extraction of the coordintes from the image data
os.chdir('/tum_nrp/rlmodel/image_processing/src/')
from custom_dataset import CMLRPiDataset
from transform_nonzeros import non_zeros
os.chdir('/tum_nrp/rlmodel/sb3')

# dataset and dataloaders
from torch.utils.data import Dataset, DataLoader

# transforms for the transformation of the custom dataset
try:
    from torchvision import transforms
except ModuleNotFoundError:
    !pip install torchvision
    from torchvision import transforms

In [ ]:
# path to the folder of the dataset 
DATASET_PATH = "/tum_nrp/rlmodel/image_processing/data/"
# path to the folder where models are saved
CHECKPOINT_PATH = "/tum_nrp/rlmodel/image_processing/saved_models/"

# number of workers for the dataloader
num_workers = 0
batch_size = 1 # has to be one!

# width and height of the table in the images captured by the top mounted camera
width = 120
height = 116

# image transformations
transform=transforms.Compose([non_zeros(float(width), float(height))])    

# create the dataset
dataset = CMLRPiDataset(
    root_dir=DATASET_PATH,
    data_pi_name ='training_data.pt',
    mask_pi_name = 'no_cylinder.pt',
    threshold=200,
    transform=transform
)

# retrive the number of samples in the dataset 
dataset_len = dataset.__len__()


# split the dataset in test train and valdatuion
train_set, test_set = torch.utils.data.random_split(dataset, [int(dataset.__len__()*0.7),  
                                                                (dataset.__len__()-int(dataset.__len__()*0.7))])

# definition of the corresponding data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=num_workers)

# sanity check

# retrive the first batch of the training data
dataiter = iter(train_loader)

error_x = 0
error_y = 0

# caculate the average error of the detected x and y value for the fist batch
for _ in range(len(train_loader)):
    for i in range(batch_size):
        cyl_loc, labels = dataiter.next()
        error_x += abs((labels[i][0])-(cyl_loc[i][0]))
        error_y += abs((labels[i][1])-(cyl_loc[i][1]))
print(error_x/(batch_size*len(train_loader)), error_y/(batch_size*len(train_loader)))

# Parameters

In [ ]:
params = {
    "VERBOSE": 1,
    "SETTING": 'reduced4+',                 # 'reduced', 'reduced2', 'reduced3', 'reduced3+', 'reduced4', 'reduced4+', 'full'
    "OBJ_SPACE_LOW": np.array([-0.92, -0.51, 0.58, -0.44, -0.48, 0, -np.pi/2, -np.pi/2, -0.001, -np.pi/2, -0.001, -np.pi]), # observation-space (ee-pos, cyl-pos, joints)
    "OBJ_SPACE_HIGH": np.array([0.92, 1.32, 2.07, 0.48, 0.44, 1.12, np.pi/2, 0.001, np.pi, np.pi/2, np.pi, np.pi]),
    "SPACE_NORM": 1,                   #  1 -> yes, 0 -> no (normalize the action and observation space)
    "CYLINDER": 'no',         # 'no'(no augmentation), fix', 'semi_random', 'semi_random_sides', 'half_table', '3/4-table', '7/8-table', 'whole_table'
    "BUFFER_SIZE": 2000,
    "THRESHOLD": 0.2,                  # initial treshold
    "THRESHOLD_SCHEDULING": 1,         # 1-> yes, 0-> no
    "MIN_THRESHOLD": 0.02,
    "REWARD_TYPE": 'dense',            # 'sparse', 'dense', 'extra_dense'
    "LEARNING_STARTS": 100,            # number of random movements before learning starts,#
    "TOGGLE_REWARD": 0,
    "STEPS": 10000,                     # number of steps while training (=num_episodes when MAX_EPISODE_LEGTH is 1)
    "MAX_EPISODE_LENGTH": 1,           # 'None' (no limit) or value 
    "EXPLORATION": 1,                  # just let it on 1 and ignore it
    "USE_HER": 1,                      # 1-> yes, 0-> no
    "GLOBAL_STEPPER": 0, 
    "EVALUATION_STEPS": 500,           # number of evaluation steps per investigates treshold (x4)
    "EVALS": [0.10, 0.07, 0.05, 0.03],  # here, the list MUST contain always 4 tresholds for evaluation
    "BATCH_SIZE": 100,
    "ACTION_NOISE": None,
    "LR": 1e-3,
# image specific
    "img_pi_data": 1,
    "train_loader": train_loader,
    "test_loader": test_loader,
# td3 specific
    "TARGET_POLICY_NOISE": 0.2,
    "TARGET_NOISE_CLIP": 0.5}

# Continue Training

This part enables us to continue the training.
(e.g. when results are likely to improve with some more episodes).

In [ ]:
##############
# load model #
##############
exp = eaw.ExperimentWrapper()
load_file_name = "TD3_0.2_dense_reduced4+_10000_20210729-123802"
writer = SummaryWriter(log_dir=f"tensorboard_log/{load_file_name}/") # set up tensorboard storage

# test if simulation can be reached
server_id = exp.client.test()
if server_id:
    print("Simulation is available, id: ", server_id)
else:
    print("Simulation NOT available")

# make a new environment
if params["USE_HER"] == 1:
    env = SimGoalEnv(exp, params, writer)
else:
    env = SimEnv(exp, params, writer)
    
# check env
env.set_eval(ev=True)
# env_checker.check_env(env)

model=TD3.load('./saved_models/'+load_file_name, env=env)

Skip the next cell if you just want to evaluate the model.

In [ ]:
#####################
# continue training # 
#####################

# make adjustments
params["STEPS"] = 2000 

# set up logging
exp = eaw.ExperimentWrapper()
server_id = exp.client.test()
if server_id:
    print("Simulation is available, id: ", server_id)
else:
    print("Simulation NOT available")
timestr = time.strftime("%Y%m%d-%H%M%S")
file_name = f"Leander_TD3_{params['THRESHOLD']}_{params['REWARD_TYPE']}_{params['SETTING']}_{params['STEPS']}_{timestr}"
writer = SummaryWriter(log_dir=f"tensorboard_log/{file_name}/") # set up tensorboard storage

# make a new environment
if params["USE_HER"] == 1:
    env = SimGoalEnv(exp, params, writer)
else:
    env = SimEnv(exp, params, writer)
    
# check env
# env_checker.check_env(env)
model=TD3.load('./saved_models/'+load_file_name, env=env)

# change env in model
model.set_env(env)
model.tensorboard_log=f"tensorboard_log/{file_name}/"

if params["USE_HER"] == 1:
    # start training
    train(model, env, params)
else:
    # start training (without train method)
    model.learn(params["STEPS"])

model.save('./saved_models/'+file_name)
model.get_parameters()

In [ ]:
##############
# save model #
##############
model.save('./saved_models/'+file_name)
model.get_parameters()

In [ ]:
####################
# model evaluation #
####################

# disable plotting while evaluation because of some issues with inconsistent lengths
env.set_eval(ev=True)

evaluate3(model, env, params, writer, max_threshold=max(params["EVALS"]))